## Step 1: load datasets
Below we generate some toy datasets using generate_toy_datasets() as defined in utils.py. User can load their own survival datasets into "datasets", which should be a list of (X, time, event) tuples, where X, time, and event are the design matrix, survival time and event vectors for a given dataset. 

In [1]:
from utils import generate_toy_datasets

n_datasets = 10 # generate 10 datasets
n_min, n_max = 100, 200 # number of samples in each dataset is a random integer between 100 and 200
n_features = 10000 # number of features is 10000
datasets = generate_toy_datasets(n_datasets, n_min, n_max, n_features)

## Step 2 (optional): feature transformation
If necessary, we can first preprocess X so that it is standardized. We provide in preprocessing.py two basic types of feature transformation functions:
- __rank_transform()__: transform features of each sample into normalized ranks
- __zscore_transform()__: transform each feature to be zero mean and unit std across samples

We can then wrap them in a FeatureTransformer object which defines the fit_transform method for our "datasets" list

In [2]:
from preprocessing import rank_transform, FeatureTransformer

feature_transformer = FeatureTransformer(rank_transform)
datasets_transformed = feature_transformer.fit_transform(datasets)

## Step 3: split into training and testing
We provide train_test_split() in utils.py to split "datasets" list in a stratified way. That is, each dataset in "datasets" is split according to test_size

In [3]:
from utils import train_test_split

datasets_train, datasets_test = train_test_split(datasets_transformed, test_size=0.2)

## Step 4: precompute index pairs
For each dataset in the datasets list, we precompute a (idx1, idx2) tuple, where idx1 and idx2 are two equal length index vectors satisfying time[idx1] < time[idx2] and event[idx1] == 0. This information is needed when defining the loss function for training.

In [4]:
from utils import get_index_pairs

index_pairs_tr = get_index_pairs(datasets_train)
index_pairs_te = get_index_pairs(datasets_test)

## Step 5 (optional): feature selection
We can additionally perform a feature selection step to reduce the number of features before model training. In feature_selection.py we provide a feature selection method based on concordance index as commonly used to characterize the feature's correlation with survival. 

Also note that our feature selection for multiple datasets is based on meta-analysis. The concordance index is calculated for each dataset and combined into a meta-score based on the size of the dataset. This is done by wrapping the score function in a SelectKBestMeta object which defines the fit and transform function. 

In [5]:
from feature_selection import concordance_index, SelectKBestMeta

topK = 1024 # select top 1024 features
feature_selector = SelectKBestMeta(concordance_index, topK)
feature_selector.fit(datasets_train)
datasets_train_new = feature_selector.transform(datasets_train)
datasets_test_new = feature_selector.transform(datasets_test)

## Step 6: user defined keras model
This is the core input required of the user. Below we provide a simple fully-connected network with 3 hidden layers. Note that there is no need to apply any activation function to the input layer. We are building a survival regression model!

In [6]:
from keras.models import Model
from keras.layers import Input, Dense, Activation, Dropout
import keras.backend as K


def build_model(feature_dim):
    '''
    Define a callable keras model yourself
    model input should be a (None, feature_dim) tensor,
    model output should be a (None, 1) tensor
    '''
    x = Input(shape=(feature_dim,))
    #--------START OF USER CODE-------
    a0 = Dropout(0.3)(x)
    z1 = Dense(units=1024, activation=None)(a0)
    a1 = Activation(activation='elu')(z1)
    a1 = Dropout(0.5)(a1)
    z2 = Dense(units=1024, activation=None)(a1)
    a2 = Activation(activation='elu')(z2)
    a2 = Dropout(0.5)(a2)
    y = Dense(units=1, activation=None)(a2)
    #--------END OF USER CODE-------
    
    model = Model(inputs=x, outputs=y)
    return model

Using TensorFlow backend.


## Step 7: build tensorflow graph

In [7]:
import tensorflow as tf

tf.reset_default_graph()

with tf.name_scope('input'):
    X = tf.placeholder(dtype=tf.float32, shape=[None, topK], name='X')
    idx1 = tf.placeholder(dtype=tf.int32, shape=[None, ])
    idx2 = tf.placeholder(dtype=tf.int32, shape=[None, ])
    
with tf.name_scope('model'):
    model = build_model(topK)
    
with tf.name_scope('output'):
    y_pred = model(X)
    y1 = tf.gather(y_pred, idx1)
    y2 = tf.gather(y_pred, idx2)
    
with tf.name_scope('metrics'):
    loss = tf.reduce_mean(tf.maximum(1+y1-y2, 0.0)) # alternatively: loss = tf.reduce_mean(tf.log(1+tf.exp(y1-y2)))
    ci = tf.reduce_mean(tf.cast(y1<y2, tf.float32), name='c_index')
    
with tf.name_scope('train'):
    optimizer = tf.train.AdamOptimizer(learning_rate=0.001)
    train_op = optimizer.minimize(loss)

C:\Users\cuiyi\Anaconda3\envs\tensorflow\lib\site-packages\tensorflow\python\ops\gradients_impl.py:93: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


## Step 8: run model training

In [8]:
init = tf.global_variables_initializer()
epochs = 1000
sess = tf.Session()

## a function just to make model eval eaiser
def total_loss_ci(datasets, index_pairs):
    loss_total = 0
    ci_total = 0
    n_total = 0
    for i, (X_batch, _, _) in enumerate(datasets):
        idx1_batch, idx2_batch = index_pairs[i]
        loss_batch, ci_batch = sess.run([loss, ci], feed_dict={X:X_batch, idx1:idx1_batch, idx2:idx2_batch, K.learning_phase():0})
        loss_total += len(idx1_batch)*loss_batch
        ci_total += len(idx1_batch)*ci_batch
        n_total += len(idx1_batch)
    loss_total /= n_total
    ci_total /= n_total
    return loss_total, ci_total

sess.run(init)
for epoch in range(epochs):
    for i, (X_batch, _, _) in enumerate(datasets_train_new):
        idx1_batch, idx2_batch = index_pairs_tr[i]
        sess.run(train_op, feed_dict={X:X_batch, idx1:idx1_batch, idx2:idx2_batch, K.learning_phase():1})
    if epoch%100==0:
        loss_tr, ci_tr = total_loss_ci(datasets_train_new, index_pairs_tr)
        loss_te, ci_te = total_loss_ci(datasets_test_new, index_pairs_te)
        print('Epoch {0}: loss_tr={1:5.4f}, ci_tr={2:5.4f}, loss_te={3:5.4f}, ci_te={4:5.4f}'.format(epoch, loss_tr, ci_tr, loss_te, ci_te))

sess.close()

Epoch 0: loss_tr=0.3691, ci_tr=0.8500, loss_te=1.1415, ci_te=0.4619
Epoch 100: loss_tr=0.1482, ci_tr=0.9556, loss_te=1.0740, ci_te=0.4899
Epoch 200: loss_tr=0.1154, ci_tr=0.9634, loss_te=1.1546, ci_te=0.4838
Epoch 300: loss_tr=0.1026, ci_tr=0.9734, loss_te=1.1217, ci_te=0.4742
Epoch 400: loss_tr=0.0716, ci_tr=0.9818, loss_te=1.1532, ci_te=0.4834
Epoch 500: loss_tr=0.0585, ci_tr=0.9861, loss_te=1.1448, ci_te=0.4869
Epoch 600: loss_tr=0.0524, ci_tr=0.9881, loss_te=1.1674, ci_te=0.4829
Epoch 700: loss_tr=0.0501, ci_tr=0.9905, loss_te=1.1148, ci_te=0.4921
Epoch 800: loss_tr=0.0444, ci_tr=0.9927, loss_te=1.1157, ci_te=0.4764
Epoch 900: loss_tr=0.0338, ci_tr=0.9944, loss_te=1.1923, ci_te=0.4663


This model achieves an almost perfect performance on the training dataset but not so on the testing dataset. This is expected since our simulated datasets are just randomly generated and there is nothing to learn (it'll be surprising if it does learn anything useful...). You can provide your own dataset and design your own model and check if it also works on testing dataset. 